<a href="https://colab.research.google.com/github/Petersonp/FitBot/blob/main/FitBot_NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers[torch] -U
!pip install accelerate==0.20.3

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import json
from transformers import BertTokenizerFast, BertForTokenClassification
from transformers import Trainer, TrainingArguments
import numpy as np
import pandas as pd
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split

In [ ]:
# Tag to index mapping
label_map = {'O': 0,
             'B-MealType': 1,
             'I-MealType': 2,
             'B-Food': 3,
             'I-Food': 4,
             'B-Dietary': 5,
             'I-Dietary': 6,
             'B-Nutrient': 7,
             'I-Nutrient': 8,
             'B-Muscle': 9,
             'I-Muscle': 10,
             'B-MuscleGroups': 11,
             'I-MuscleGroups': 12,
             'B-Time': 13,
             'I-Time': 14,
             'B-Workout': 15,
             'I-Workout': 16,
             'B-Goal': 17,
             'I-Goal': 18,
             'B-Age': 19,
             "B-Equipment": 20,
             "B-Number":21,
             "I-Number":22,
             "B-Weight": 23,
             "I-Weight": 24,
             "B-Duration":25,
             "I-Duration":26}

# Load the tokenizer and model
num_labels=len(label_map)

tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
model = BertForTokenClassification.from_pretrained('bert-base-uncased', num_labels=num_labels)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
with open('/content/drive/MyDrive/23-24/CS 4701/ner.json', 'r') as f:
    data = json.load(f)

In [ ]:
sentences = data['sentences']
tags = data['tags']
train_sentences, val_sentences, train_tags, val_tags = train_test_split(sentences, tags, test_size=0.2)

In [ ]:
def tokenize_and_align_labels(sentences, tags, tokenizer, label_map):
    tokenized_inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt", is_split_into_words=True)
    labels = []
    for i, label in enumerate(tags):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)  # Special token
            elif word_idx != previous_word_idx:
                label_ids.append(label_map[label[word_idx]])
            else:
                label_ids.append(label_map[label[word_idx]])
            previous_word_idx = word_idx
        labels.append(label_ids)
    return tokenized_inputs, labels

train_tokenized_inputs, train_labels = tokenize_and_align_labels(train_sentences, train_tags, tokenizer, label_map)
val_tokenized_inputs, val_labels = tokenize_and_align_labels(val_sentences, val_tags, tokenizer, label_map)


In [ ]:
class NERDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = NERDataset(train_tokenized_inputs, train_labels)
val_dataset = NERDataset(val_tokenized_inputs, val_labels)

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',           # output directory for model checkpoints
    num_train_epochs=10,               # number of training epochs
    per_device_train_batch_size=5,   # batch size for training
    per_device_eval_batch_size=10,    # batch size for evaluation
    warmup_steps=500,                 # warmup steps for learning rate scheduler
    weight_decay=0.01,                # strength of weight decay
    logging_dir='./logs',             # directory for storing logs
    evaluation_strategy='epoch',
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,  # Optionally, you can also provide an evaluation dataset
)

trainer.train()

<ipython-input-8-694e13bc35dd>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Epoch,Training Loss,Validation Loss
1,No log,1.994096
2,No log,1.248016
3,No log,0.831100
4,No log,0.625842
5,No log,0.608836
6,No log,0.565891
7,No log,0.620930
8,No log,0.604421
9,0.802000,0.629188
10,0.802000,0.674521


<ipython-input-8-694e13bc35dd>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


TrainOutput(global_step=590, training_loss=0.6873188487554001, metrics={'train_runtime': 57.1954, 'train_samples_per_second': 50.878, 'train_steps_per_second': 10.316, 'total_flos': 29708805639600.0, 'train_loss': 0.6873188487554001, 'epoch': 10.0})

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
def predict_entities(sentence, model, tokenizer, label_map):
    # Ensure model is in evaluation mode
    model.eval()

    # Tokenize the sentence
    inputs = tokenizer(sentence, truncation=True, padding=True, return_tensors="pt")

    # Move the inputs to the same device as the model
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    # Predict
    with torch.no_grad():
        outputs = model(**inputs).logits

    # Find the predicted labels
    predictions = torch.argmax(outputs, dim=2)

    # Convert the predictions to label names
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0].to('cpu'))
    label_indices = predictions[0].to('cpu').numpy()
    labels = [list(label_map.keys())[list(label_map.values()).index(i)] for i in label_indices]

    # Pair each token with its label
    token_label_pairs = list(zip(tokens, labels))

    return token_label_pairs
def clean_predictions(token_label_pairs):
    # Filter out special tokens and subword tokens
    return [(token, label) for token, label in token_label_pairs if token not in ["[CLS]", "[SEP]", "[PAD]"] and not token.startswith("##")]


In [ ]:
test_sentence = "create a 30 min workout plan for bicep"
predictions = predict_entities(test_sentence, model, tokenizer, label_map)
cleaned_predictions = clean_predictions(predictions)
print(cleaned_predictions)

[('create', 'O'), ('a', 'O'), ('30', 'B-Number'), ('min', 'B-Time'), ('workout', 'O'), ('plan', 'O'), ('for', 'O'), ('bi', 'B-Muscle')]


In [ ]:
model.save_pretrained('/content/drive/MyDrive/23-24/CS 4701/NER Trained Model')
tokenizer.save_pretrained('/content/drive/MyDrive/23-24/CS 4701/NER Trained Model')

('/content/drive/MyDrive/23-24/CS 4701/NER Trained Model/tokenizer_config.json',
 '/content/drive/MyDrive/23-24/CS 4701/NER Trained Model/special_tokens_map.json',
 '/content/drive/MyDrive/23-24/CS 4701/NER Trained Model/vocab.txt',
 '/content/drive/MyDrive/23-24/CS 4701/NER Trained Model/added_tokens.json',
 '/content/drive/MyDrive/23-24/CS 4701/NER Trained Model/tokenizer.json')